In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.ensemble import IsolationForest

In [ ]:
# Load the data
df = pd.read_csv('/kaggle/input/dementia/vitals_and_dementia_ratings.csv')

In [ ]:
# Convert Timestamp to datetime
df['Timestamp'] = pd.to_datetime(df['Timestamp'])

In [ ]:
# Check for missing values and handle them
print(df.isnull().sum())
df = df.dropna()  # Drop rows with missing values

In [ ]:
# Sort the dataframe by Patient ID and Timestamp
df = df.sort_values(['Patient ID', 'Timestamp'])

In [ ]:
# Calculate correlation matrix
corr_matrix = df.corr()

In [ ]:
# Plot heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm')
plt.title('Correlation Heatmap of Vital Signs and CDR')
plt.show()

In [ ]:
# Resample data to daily frequency (adjust as needed)
resampled_dfs = []
for patient_id, group in df.groupby('Patient ID'):
    resampled = group.resample('1h', on='Timestamp').mean().reset_index()
    resampled['Patient ID'] = patient_id
    resampled_dfs.append(resampled)

df_resampled = pd.concat(resampled_dfs, ignore_index=True)

In [ ]:
df.head()

In [ ]:
from sklearn.impute import SimpleImputer

In [ ]:
# If there are still missing values, use SimpleImputer
imputer = SimpleImputer(strategy='mean')
columns_to_impute = ['SPO2', 'Blood Pressure Systolic', 'Blood Pressure Diastolic', 'Temperature', 'Heart Rate', 'Clinical Dementia Rating']
df_resampled[columns_to_impute] = imputer.fit_transform(df_resampled[columns_to_impute])

In [ ]:
df_resampled.describe()

In [ ]:
# Calculate correlation matrix
corr_matrix = df_resampled.corr()

In [ ]:
# Plot heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm')
plt.title('Correlation Heatmap of Vital Signs and CDR')
plt.show()

# **Trend and seasonality analysis:**

In [ ]:
def plot_seasonality_trend(data, column, patient_id):
    plt.figure(figsize=(12, 8))

    # Perform seasonal decomposition
    result = seasonal_decompose(data[column], model='additive', period=24)  # Adjust period as needed

    # Plot original data
    plt.subplot(411)
    plt.plot(data['Timestamp'], data[column])
    plt.title(f'Seasonality and Trend Analysis for Patient {patient_id} - {column}')
    plt.ylabel('Original')

    # Plot trend
    plt.subplot(412)
    plt.plot(data['Timestamp'], result.trend)
    plt.ylabel('Trend')

    # Plot seasonal
    plt.subplot(413)
    plt.plot(data['Timestamp'], result.seasonal)
    plt.ylabel('Seasonal')

    # Plot residual
    plt.subplot(414)
    plt.plot(data['Timestamp'], result.resid)
    plt.ylabel('Residual')

    plt.tight_layout()
    plt.show()

In [ ]:
# Analyze seasonality and trend for each patient and vital sign
vital_signs = ['SPO2', 'Blood Pressure Systolic', 'Blood Pressure Diastolic', 'Temperature', 'Heart Rate','Clinical Dementia Rating']

In [ ]:
for patient_id, group in df_resampled.groupby('Patient ID'):
    for vital in vital_signs:
        plot_seasonality_trend(group, vital, patient_id)

In [ ]:
# Calculate overall trend
def calculate_trend(data, column):
    x = np.arange(len(data))
    y = data[column].values
    slope, _ = np.polyfit(x, y, 1)
    return slope

In [ ]:
trends = df_resampled.groupby('Patient ID').apply(lambda x: pd.Series({
    vital: calculate_trend(x, vital) for vital in vital_signs
}))

print("Overall trends for each patient:")
print(trends)

In [ ]:
def detect_anomalies(data, column):
    clf = IsolationForest(contamination=0.1, random_state=42)
    anomalies = clf.fit_predict(data[[column]])
    return pd.Series(anomalies == -1, index=data.index)

In [ ]:
for vital in vital_signs:
    df_resampled[f'{vital}_anomaly'] = df_resampled.groupby('Patient ID').apply(
        lambda x: detect_anomalies(x, vital)
    ).reset_index(level=0, drop=True)

In [ ]:
def plot_anomalies(data, column, patient_id):
    plt.figure(figsize=(12, 6))
    plt.plot(data['Timestamp'], data[column], label='Normal')

    anomaly_column = f'{column}_anomaly'
    if anomaly_column in data.columns:
        anomalies = data[data[anomaly_column]]
        plt.scatter(anomalies['Timestamp'], anomalies[column], color='red', label='Anomaly')

    plt.title(f'Anomaly Detection for Patient {patient_id} - {column}')
    plt.xlabel('Timestamp')
    plt.ylabel(column)
    plt.legend()
    plt.show()

In [ ]:
for patient_id, group in df_resampled.groupby('Patient ID'):
    for vital in vital_signs:
        plot_anomalies(group, vital, patient_id)

# **SARIMAX Model:**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error
from math import sqrt

In [ ]:
def build_sarimax_model(patient_data, target='Clinical Dementia Rating', exog_features=['SPO2', 'Blood Pressure Systolic', 'Blood Pressure Diastolic', 'Temperature', 'Heart Rate']):
    # Prepare the data
    y = patient_data[target]
    X = patient_data[exog_features]

    # Split data into train and test sets
    train_size = int(len(y) * 0.8)
    y_train, y_test = y[:train_size], y[train_size:]
    X_train, X_test = X[:train_size], X[train_size:]

    # Fit SARIMAX model
    # Note: You may need to adjust the order and seasonal_order parameters
    model = SARIMAX(y_train, exog=X_train, order=(1, 1, 1), seasonal_order=(1, 1, 1, 24))
    results = model.fit()

    # Print model summary
    print(results.summary())

    # Make predictions
    forecast = results.get_forecast(steps=len(y_test), exog=X_test)
    forecast_mean = forecast.predicted_mean

    # Calculate RMSE
    rmse = sqrt(mean_squared_error(y_test, forecast_mean))
    print(f'RMSE: {rmse}')

    # Plot the results
    plt.figure(figsize=(12, 6))
    plt.plot(y_train.index, y_train, label='Training Data')
    plt.plot(y_test.index, y_test, label='True Test Data')
    plt.plot(y_test.index, forecast_mean, label='Predictions')
    plt.title(f'SARIMAX Forecast (RMSE: {rmse:.2f})')
    plt.legend()
    plt.show()

    return results, rmse

In [ ]:
# Build SARIMAX models for each patient
sarimax_models = {}
for patient in df_resampled['Patient ID'].unique():
    print(f"\nBuilding SARIMAX model for Patient {patient}")
    patient_data = df_resampled[df_resampled['Patient ID'] == patient].set_index('Timestamp')
    model, rmse = build_sarimax_model(patient_data)
    sarimax_models[patient] = {'model': model, 'rmse': rmse}

In [ ]:
# Print summary of all models
for patient, model_info in sarimax_models.items():
    print(f"\nPatient {patient}:")
    print(f"RMSE: {model_info['rmse']:.2f}")

# **LSTM model:**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam
from math import sqrt

In [ ]:
def create_sequences(data, target, seq_length):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:(i + seq_length)])
        y.append(target[i + seq_length])
    return np.array(X), np.array(y)

In [ ]:
def build_lstm_model(patient_data, seq_length=7):
    # Define features and target
    features = ['SPO2', 'Blood Pressure Systolic', 'Blood Pressure Diastolic', 'Temperature', 'Heart Rate']
    target = 'Clinical Dementia Rating'

    # Normalize the data
    scaler_X = MinMaxScaler(feature_range=(0, 1))
    scaler_y = MinMaxScaler(feature_range=(0, 1))

    scaled_features = scaler_X.fit_transform(patient_data[features])
    scaled_target = scaler_y.fit_transform(patient_data[[target]])

    # Create sequences
    X, y = create_sequences(scaled_features, scaled_target, seq_length)

    # Split into train and test sets
    train_size = int(len(X) * 0.8)
    X_train, X_test = X[:train_size], X[train_size:]
    y_train, y_test = y[:train_size], y[train_size:]

    # Build the LSTM model
    model = Sequential([
        LSTM(50, activation='relu', input_shape=(seq_length, len(features))),
        Dense(1)
    ])
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')

    # Train the model
    history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2, verbose=0)

    # Make predictions
    train_predict = model.predict(X_train)
    test_predict = model.predict(X_test)

    # Invert predictions to original scale
    train_predict = scaler_y.inverse_transform(train_predict)
    y_train = scaler_y.inverse_transform(y_train)
    test_predict = scaler_y.inverse_transform(test_predict)
    y_test = scaler_y.inverse_transform(y_test)

    # Calculate RMSE and R2 score
    train_rmse = sqrt(mean_squared_error(y_train, train_predict))
    test_rmse = sqrt(mean_squared_error(y_test, test_predict))
    train_r2 = r2_score(y_train, train_predict)
    test_r2 = r2_score(y_test, test_predict)

    print(f'Train RMSE: {train_rmse:.3f}')
    print(f'Test RMSE: {test_rmse:.3f}')
    print(f'Train R2 Score: {train_r2:.3f}')
    print(f'Test R2 Score: {test_r2:.3f}')

    # Plot results
    plt.figure(figsize=(12, 6))
    plt.plot(patient_data.index[seq_length:train_size+seq_length], y_train, label='Train Actual')
    plt.plot(patient_data.index[seq_length:train_size+seq_length], train_predict, label='Train Predict')
    plt.plot(patient_data.index[train_size+seq_length:], y_test, label='Test Actual')
    plt.plot(patient_data.index[train_size+seq_length:], test_predict, label='Test Predict')
    plt.title('LSTM Model: Clinical Dementia Rating Prediction')
    plt.legend()
    plt.show()

    # Plot loss
    plt.figure(figsize=(12, 6))
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Model Loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend()
    plt.show()

    return model, train_rmse, test_rmse, train_r2, test_r2

In [ ]:
# Build LSTM models for each patient
lstm_models = {}
for patient in df_resampled['Patient ID'].unique():
    print(f"\nBuilding LSTM model for Patient {patient}")
    patient_data = df_resampled[df_resampled['Patient ID'] == patient].set_index('Timestamp')
    model, train_rmse, test_rmse, train_r2, test_r2 = build_lstm_model(patient_data)
    lstm_models[patient] = {
        'model': model,
        'train_rmse': train_rmse,
        'test_rmse': test_rmse,
        'train_r2': train_r2,
        'test_r2': test_r2
    }

In [ ]:
# Print summary of all models
for patient, model_info in lstm_models.items():
    print(f"\nPatient {patient}:")
    print(f"Train RMSE: {model_info['train_rmse']:.3f}")
    print(f"Test RMSE: {model_info['test_rmse']:.3f}")
    print(f"Train R2 Score: {model_info['train_r2']:.3f}")
    print(f"Test R2 Score: {model_info['test_r2']:.3f}")

In [ ]:
pip install prophet

In [ ]:
pip install catboost

In [ ]:
pip install dask

In [ ]:
pip install lightgbm  xgboost

# **Boosting Models:**

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Assuming df_resampled is your preprocessed dataframe
df = df_resampled.copy()

In [ ]:
# Define features and target
target = 'Clinical Dementia Rating'
features = ['SPO2', 'Blood Pressure Systolic', 'Blood Pressure Diastolic', 'Temperature', 'Heart Rate']

In [ ]:
# Function to evaluate models
def evaluate_model(y_true, y_pred, model_name):
    mse = mean_squared_error(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_true, y_pred)

    return {
        'Model': model_name,
        'MSE': mse,
        'MAE': mae,
        'RMSE': rmse,
        'R-squared': r2
    }

In [ ]:
# Function to plot results
def plot_results(y_true, y_pred, timestamps, model_name, patient_id):
    plt.figure(figsize=(12, 6))
    plt.plot(timestamps, y_true, label='Actual')
    plt.plot(timestamps, y_pred, label='Predicted')
    plt.title(f'{model_name}: Actual vs Predicted Dementia Rating for Patient {patient_id}')
    plt.xlabel('Timestamp')
    plt.ylabel('Dementia Rating')
    plt.legend()
    plt.show()

In [ ]:
# Initialize lists to store results
all_results = []
feature_importance = {model: [] for model in ['CatBoost', 'LightGBM', 'XGBoost']}

In [ ]:
# Iterate over each patient
for patient_id, patient_data in df.groupby('Patient ID'):
    print(f"Processing Patient {patient_id}")

    # Prepare data for modeling
    X = patient_data[features]
    y = patient_data[target]

    # Split data into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # CatBoost model
    catboost_model = CatBoostRegressor(iterations=1000, learning_rate=0.1, depth=6, random_state=42)
    catboost_model.fit(X_train, y_train, verbose=False)
    catboost_pred = catboost_model.predict(X_test)
    all_results.append({**evaluate_model(y_test, catboost_pred, 'CatBoost'), 'Patient ID': patient_id})
    feature_importance['CatBoost'].append(catboost_model.feature_importances_)
    plot_results(y_test, catboost_pred, patient_data['Timestamp'][-len(y_test):], 'CatBoost', patient_id)

    # LightGBM model
    lgbm_model = LGBMRegressor(n_estimators=1000, learning_rate=0.1, max_depth=6, random_state=42)
    lgbm_model.fit(X_train, y_train)
    lgbm_pred = lgbm_model.predict(X_test)
    all_results.append({**evaluate_model(y_test, lgbm_pred, 'LightGBM'), 'Patient ID': patient_id})
    feature_importance['LightGBM'].append(lgbm_model.feature_importances_)
    plot_results(y_test, lgbm_pred, patient_data['Timestamp'][-len(y_test):], 'LightGBM', patient_id)

    # XGBoost model
    xgb_model = XGBRegressor(n_estimators=1000, learning_rate=0.1, max_depth=6, random_state=42)
    xgb_model.fit(X_train, y_train)
    xgb_pred = xgb_model.predict(X_test)
    all_results.append({**evaluate_model(y_test, xgb_pred, 'XGBoost'), 'Patient ID': patient_id})
    feature_importance['XGBoost'].append(xgb_model.feature_importances_)
    plot_results(y_test, xgb_pred, patient_data['Timestamp'][-len(y_test):], 'XGBoost', patient_id)

In [ ]:
# Convert results to DataFrame
results_df = pd.DataFrame(all_results)

In [ ]:
# Print average performance for each model
print(results_df.groupby('Model').mean())

In [ ]:
# Plot average MSE for each model
avg_mse = results_df.groupby('Model')['MSE'].mean()
plt.figure(figsize=(10, 6))
avg_mse.plot(kind='bar')
plt.title('Average MSE by Model')
plt.ylabel('MSE')
plt.show()

In [ ]:
# Plot average feature importance
for model in ['CatBoost', 'LightGBM', 'XGBoost']:
    avg_importance = np.mean(feature_importance[model], axis=0)
    plt.figure(figsize=(10, 6))
    plt.bar(features, avg_importance)
    plt.title(f'Average Feature Importance - {model}')
    plt.xlabel('Features')
    plt.ylabel('Importance')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

In [ ]:
# Plot MSE distribution for each model
plt.figure(figsize=(12, 6))
results_df.boxplot(column='MSE', by='Model')
plt.title('MSE Distribution by Model')
plt.suptitle('')  # This removes the automatic suptitle added by boxplot
plt.show()

In [ ]:
results_df.head(30)

In [ ]:
results_df.tail()

# **Prophet Model:**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from prophet import Prophet
from prophet.plot import plot_plotly, plot_components_plotly
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from tqdm import tqdm

In [ ]:
target = 'Clinical Dementia Rating'
exog_vars = ['SPO2', 'Blood Pressure Systolic', 'Blood Pressure Diastolic', 'Temperature', 'Heart Rate']

In [ ]:
# Function to prepare data for Prophet
def prepare_prophet_data(data, target, exog_vars):
    df = data.copy()
    df = df.rename(columns={'Timestamp': 'ds', target: 'y'})
    return df[['ds', 'y'] + exog_vars]

# Function to evaluate Prophet model
def evaluate_prophet(train, test, exog_vars):
    model = Prophet()
    for var in exog_vars:
        model.add_regressor(var)

    model.fit(train)

    future = test[['ds'] + exog_vars]
    forecast = model.predict(future)

    y_true = test['y'].values
    y_pred = forecast['yhat'].values

    mse = mean_squared_error(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_true, y_pred)

    return model, mse, mae, rmse, r2

In [ ]:
#Prepare data and evaluate model for each patient
results = {}
for patient_id, group in tqdm(df_resampled.groupby('Patient ID')):
    data = prepare_prophet_data(group, target, exog_vars)

    # Split data into train and test sets
    train_size = int(len(data) * 0.8)
    train, test = data[:train_size], data[train_size:]

    model, mse, mae, rmse, r2 = evaluate_prophet(train, test, exog_vars)

    results[patient_id] = {
        'model': model,
        'mse': mse,
        'mae': mae,
        'rmse': rmse,
        'r2': r2
    }

In [ ]:
# Print average metrics across all patients
avg_mse = np.mean([res['mse'] for res in results.values()])
avg_mae = np.mean([res['mae'] for res in results.values()])
avg_rmse = np.mean([res['rmse'] for res in results.values()])
avg_r2 = np.mean([res['r2'] for res in results.values()])

print(f'Average Model Performance:')
print(f'MSE: {avg_mse}')
print(f'MAE: {avg_mae}')
print(f'RMSE: {avg_rmse}')
print(f'R-squared: {avg_r2}')

In [ ]:
# Plot results for a single patient (you can change the patient_id as needed)
patient_id = list(results.keys())[9]
model = results[patient_id]['model']
data = prepare_prophet_data(df_resampled[df_resampled['Patient ID'] == patient_id], target, exog_vars)

In [ ]:
future = model.make_future_dataframe(periods=24)  # Forecast for next 30 days (720 hours)
for var in exog_vars:
    future[var] = data[var].mean()  # Use mean values for exogenous variables

In [ ]:
forecast = model.predict(future)

In [ ]:
fig1 = plot_plotly(model, forecast)
fig1.update_layout(xaxis=dict(title='Date and Hour', tickformat='%Y-%m-%d %H:%M'))
fig1.show()

In [ ]:
fig2 = plot_components_plotly(model, forecast)
fig2.update_layout(xaxis=dict(title='Date and Hour', tickformat='%Y-%m-%d %H:%M'))
fig2.show()

# **Models comparison:**

In [ ]:
def compare_rmse(sarimax_models, lstm_models, boosting_results, prophet_results):
    # Extract RMSE values
    sarimax_rmse = [model_info['rmse'] for model_info in sarimax_models.values()]
    lstm_rmse = [model_info['test_rmse'] for model_info in lstm_models.values()]
    
    # For boosting models, we need to extract RMSE for each model type
    catboost_rmse = boosting_results[boosting_results['Model'] == 'CatBoost']['RMSE'].tolist()
    lightgbm_rmse = boosting_results[boosting_results['Model'] == 'LightGBM']['RMSE'].tolist()
    xgboost_rmse = boosting_results[boosting_results['Model'] == 'XGBoost']['RMSE'].tolist()
    
    prophet_rmse = [res['rmse'] for res in prophet_results.values()]

    # Create a DataFrame
    df_rmse = pd.DataFrame({
        'SARIMAX': sarimax_rmse,
        'LSTM': lstm_rmse,
        'CatBoost': catboost_rmse,
        'LightGBM': lightgbm_rmse,
        'XGBoost': xgboost_rmse,
        'Prophet': prophet_rmse
    })

    # Calculate mean RMSE for each model
    mean_rmse = df_rmse.mean()

    # Plot
    plt.figure(figsize=(12, 6))
    mean_rmse.plot(kind='bar')
    plt.title('Comparison of Mean RMSE Across Models')
    plt.ylabel('Mean RMSE')
    plt.xlabel('Models')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

    # Print mean RMSE values
    print("Mean RMSE for each model:")
    print(mean_rmse)

    return df_rmse

In [ ]:
# Call the function
rmse_comparison = compare_rmse(sarimax_models, lstm_models, results_df, results)

In [ ]:
def compare_rmse_per_patient(sarimax_models, lstm_models, boosting_results, prophet_results):
    # Initialize a dictionary to store RMSE values for each patient
    patient_rmse = {}

    # Extract RMSE values for each patient
    for patient_id in sarimax_models.keys():
        patient_rmse[patient_id] = {
            'SARIMAX': sarimax_models[patient_id]['rmse'],
            'LSTM': lstm_models[patient_id]['test_rmse'],
            'CatBoost': boosting_results[(boosting_results['Model'] == 'CatBoost') & (boosting_results['Patient ID'] == patient_id)]['RMSE'].values[0],
            'LightGBM': boosting_results[(boosting_results['Model'] == 'LightGBM') & (boosting_results['Patient ID'] == patient_id)]['RMSE'].values[0],
            'XGBoost': boosting_results[(boosting_results['Model'] == 'XGBoost') & (boosting_results['Patient ID'] == patient_id)]['RMSE'].values[0],
            'Prophet': prophet_results[patient_id]['rmse']
        }

    # Convert to DataFrame
    df_rmse = pd.DataFrame(patient_rmse).T
    df_rmse.index.name = 'Patient ID'

    # Calculate mean RMSE for each model
    mean_rmse = df_rmse.mean()

    # Plot mean RMSE comparison
    plt.figure(figsize=(12, 6))
    mean_rmse.plot(kind='bar')
    plt.title('Comparison of Mean RMSE Across Models')
    plt.ylabel('Mean RMSE')
    plt.xlabel('Models')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

    # Print mean RMSE values
    print("Mean RMSE for each model:")
    print(mean_rmse)

    # Plot heatmap of RMSE values for all patients and models
    plt.figure(figsize=(12, 8))
    sns.heatmap(df_rmse, annot=True, cmap='YlOrRd', fmt='.2f')
    plt.title('RMSE Comparison Across Models and Patients')
    plt.tight_layout()
    plt.show()

    # Find the best model for each patient
    df_rmse['Best Model'] = df_rmse.idxmin(axis=1)
    
    # Print the best model for each patient
    print("\nBest model for each patient:")
    print(df_rmse['Best Model'])

    # Count the number of times each model is the best
    best_model_counts = df_rmse['Best Model'].value_counts()
    
    # Plot the count of best models
    plt.figure(figsize=(10, 6))
    best_model_counts.plot(kind='bar')
    plt.title('Count of Best Performing Models')
    plt.ylabel('Count')
    plt.xlabel('Models')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

    return df_rmse

In [ ]:
# Call the function
rmse_comparison = compare_rmse_per_patient(sarimax_models, lstm_models, results_df, results)